In [4]:
import pandas as pd
import numpy as np

from sklearn.decomposition import LatentDirichletAllocation
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.metrics import roc_auc_score

In [5]:
df_full = pd.read_csv('data/reviews_with_en.csv').drop('Unnamed: 0', axis = 1)

df = df_full[['message_en']].dropna()
df.head()

/root/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,message_en
3,I received it well before the deadline.
4,Congratulations lannister stores loved shoppin...
9,efficient device. on the website the brand of ...
12,"But a little, braking ... for the value ta Boa."
15,"Reliable seller, ok product and delivery befor..."


In [6]:
df.message_en = df.message_en.fillna(' ')

## Processing - countvectorizer

In [7]:
from nltk.corpus import stopwords
# nltk.download('stopwords')
stop_words = stopwords.words('english')
a = df['message_en']

In [76]:
CountVectorizer?

Init signature:
CountVectorizer(
    *,
    input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    analyzer='word',
    max_df=1.0,
    min_df=1,
    max_features=None,
    vocabulary=None,
    binary=False,
    dtype=<class 'numpy.int64'>,
)
Docstring:     
Convert a collection of text documents to a matrix of token counts

This implementation produces a sparse representation of the counts using
scipy.sparse.csr_matrix.

If you do not provide an a-priori dictionary and you do not use an analyzer
that does some kind of feature selection then the number of features will
be equal to the vocabulary size found by analyzing the data.

Read more in the :ref:`User Guide <text_feature_extraction>`.

Parameters
----------
input : string {'filename', 'file', 'content'}, default='content'
    If 'filename', the se

In [8]:
count_vect = CountVectorizer(min_df=0.001, stop_words=stop_words)
doc_term_matrix = count_vect.fit_transform(df['message_en'].values)

## LDA

In [9]:
LDA_liked = LatentDirichletAllocation(n_components=5, random_state=88)
LDA_liked.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=5, random_state=88)

In [10]:

for i,topic in enumerate(LDA_liked.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')


# LDA is shit - nothing interesting here

Top 10 words for topic #0:
['quality', 'time', 'well', 'delivered', 'deadline', 'recommend', 'arrived', 'great', 'good', 'product']


Top 10 words for topic #1:
['much', 'came', 'arrived', 'ok', 'office', 'right', 'post', 'expected', 'product', 'everything']


Top 10 words for topic #2:
['sent', 'return', 'two', 'delivered', '39', 'received', 'came', 'one', 'bought', 'product']


Top 10 words for topic #3:
['still', 'delivered', 'days', 'time', 'receive', 'yet', 'delivery', 'received', 'product', '39']


Top 10 words for topic #4:
['congratulations', 'always', 'satisfied', 'buy', 'recommend', 'time', 'super', 'product', 'fast', 'delivery']




## K-means

In [11]:

# Trying kmeans
from sklearn.cluster import KMeans

km = KMeans(n_clusters=3)
km.fit(doc_term_matrix)

KMeans(n_clusters=3)

In [12]:
df_cl = pd.DataFrame({'message_en':df['message_en'], 'cluster':km.predict(doc_term_matrix)})

In [13]:
df_cl.query('cluster ==1')


,message_en,cluster
38,"I&#39;m completely in love, super responsible ...",1
68,The product did not arrive within the stipulat...,1
150,"I bought the watch, unisex and sent a women&#3...",1
151,I always shop at lannister. For me it&#39;s th...,1
168,I bought the product on the 25th of February a...,1
...,...,...
99522,"I bought 3 units of this curtain, and only 1 a...",1
99531,I haven&#39;t received the blouse on delivery yet,1
99537,I&#39;m waiting for delivery.,1
99546,"The bracelet I bought is not cool, the bath wa...",1


## Biterm topic model

In [14]:
# get bigrams
from biterm.utility import vec_to_biterms

vocab = np.array(count_vect.get_feature_names())
biterms = vec_to_biterms(doc_term_matrix[:1000,:])

In [15]:

from biterm.cbtm import oBTM

btm = oBTM(num_topics=3, V=vocab)
topics = btm.fit_transform(biterms, iterations=100)

100%|██████████| 100/100 [01:57<00:00,  1.18s/it]
<ipython-input-15-0a48d6f33ee1>:4: RuntimeWarning: invalid value encountered in true_divide
  topics = btm.fit_transform(biterms, iterations=100)


In [16]:
topics.shape

(1000, 3)

In [17]:
topics

array([[1.00000000e+00, 9.78033498e-11, 9.78033498e-11],
       [1.00000000e+00, 9.78033498e-11, 9.78033498e-11],
       [1.00000000e+00, 9.78033498e-11, 9.78033498e-11],
       ...,
       [1.00000000e+00, 9.78033498e-11, 9.78033498e-11],
       [1.00000000e+00, 9.78033498e-11, 9.78033498e-11],
       [1.00000000e+00, 9.78033498e-11, 9.78033498e-11]])

In [25]:
np.nanmin(topics, axis=0,)

array([1.00000000e+00, 9.78033498e-11, 9.78033498e-11])

In [26]:
np.nanmax(topics, axis=0,)

array([1.00000000e+00, 9.78033498e-11, 9.78033498e-11])

The same predicted for each of the entries - bad model

## Movie group process - Gibbs sampling

In [28]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [33]:
import nltk
# nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer 

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatize Single Word
print(lemmatizer.lemmatize("bats"))

bat


In [36]:
sentence = "The striped bats are hanging on their feet for best"

# Tokenize: Split the sentence into words
word_list = nltk.word_tokenize(sentence)
print(word_list)
lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])

print(lemmatized_output)

['The', 'striped', 'bats', 'are', 'hanging', 'on', 'their', 'feet', 'for', 'best']
The striped bat are hanging on their foot for best


In [38]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [40]:
# Lemmatize with POS Tag
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


# 1. Init Lemmatizer
lemmatizer = WordNetLemmatizer()

# 2. Lemmatize Single Word with the appropriate POS tag
word = 'feet'
print(lemmatizer.lemmatize(word, get_wordnet_pos(word)))

# 3. Lemmatize a Sentence with the appropriate POS tag
sentence = "The striped bats are hanging on their feet for best"
print([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)])
#> ['The', 'strip', 'bat', 'be', 'hang', 'on', 'their', 'foot', 'for', 'best']

foot
['The', 'strip', 'bat', 'be', 'hang', 'on', 'their', 'foot', 'for', 'best']


In [46]:
def preprocess_one_sentence(sentence):
    
    a = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence.lower())]
    return [i for i in a if i not in stop_words]

In [47]:
preprocess_one_sentence(sentence)

['strip', 'bat', 'hang', 'foot', 'best']

In [51]:
from tqdm import tqdm
in_sent = list(df.message_en)
out_sent = [None]*len(in_sent)
for i in tqdm(range(len(out_sent))):
    out_sent[i] = preprocess_one_sentence(in_sent[i])
    

100%|██████████| 41751/41751 [01:30<00:00, 463.86it/s]


In [55]:
import string

In [58]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [60]:
for i in range(len(out_sent)):
    out_sent[i] = [sent for sent in out_sent[i] if sent not in string.punctuation]

In [61]:
out_sent[1]

['congratulation',
 'lannister',
 'store',
 'love',
 'shopping',
 'online',
 'safe',
 'practical',
 'congratulation',
 'happy',
 'easter']

In [64]:
import itertools
flatten = itertools.chain.from_iterable
temp_vocab = list(flatten(out_sent))

In [65]:
len(set(temp_vocab))

10505

In [ ]:

in_sent = list(df.message_en)#[:1000] 
out_sent = []
temp_vocab_size= []

for i in in_sent:
    temp = []
    # print(i)
    doc = nlp(i)
    for token in doc:
        if token.pos_ in ['VERB', 'ADJ', 'NOUN', 'ADV']:
            # print(token.lemma_, token.pos_)
            temp.append(token.lemma_)
            temp_vocab_size.append(token.lemma_)

        # print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #     token.shape_, token.is_alpha, token.is_stop)
    out_sent.append(set(temp))
    # print('')

In [ ]:
len(set(temp_vocab_size))
out_sent

In [31]:
list(df.message_en)

,message_en
3,I received it well before the deadline.
4,Congratulations lannister stores loved shoppin...
9,efficient device. on the website the brand of ...
12,"But a little, braking ... for the value ta Boa."
15,"Reliable seller, ok product and delivery befor..."
...,...
99985,Entregou dentro do prazo. O produto chegou em ...
99992,"O produto não foi enviado com NF, não existe v..."
99998,"Excelente mochila, entrega super rápida. Super..."
100000,Solicitei a compra de uma capa de retrovisor c...


In [68]:

# in_sent = list(df.message_en)#[:1000] 
# out_sent = []
# temp_vocab_size= []

# for i in in_sent:
#     temp = []
#     # print(i)
#     doc = nlp(i)
#     for token in doc:
#         if token.pos_ in ['VERB', 'ADJ', 'NOUN', 'ADV']:
#             # print(token.lemma_, token.pos_)
#             temp.append(token.lemma_)
#             temp_vocab_size.append(token.lemma_)

#         # print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#         #     token.shape_, token.is_alpha, token.is_stop)
#     out_sent.append(set(temp))
#     # print('')

In [69]:
from gsdmm.gsdmm.mgp import MovieGroupProcess

In [77]:
MovieGroupProcess?

Init signature: MovieGroupProcess(K=8, alpha=0.1, beta=0.1, n_iters=30)
Docstring:      <no docstring>
Init docstring:
A MovieGroupProcess is a conceptual model introduced by Yin and Wang 2014 to
describe their Gibbs sampling algorithm for a Dirichlet Mixture Model for the
clustering short text documents.
Reference: http://dbgroup.cs.tsinghua.edu.cn/wangjy/papers/KDD14-GSDMM.pdf

Imagine a professor is leading a film class. At the start of the class, the students
are randomly assigned to K tables. Before class begins, the students make lists of
their favorite films. The teacher reads the role n_iters times. When
a student is called, the student must select a new table satisfying either:
    1) The new table has more students than the current table.
OR
    2) The new table has students with similar lists of favorite movies.

:param K: int
    Upper bound on the number of possible clusters. Typically many fewer
:param alpha: float between 0 and 1
    Alpha controls the probability that a

In [78]:
mgp = MovieGroupProcess(K=30, alpha=0.1, beta=0.9, n_iters=30)

In [79]:
y = mgp.fit(out_sent,vocab_size=len(set(temp_vocab)))

In stage 0: transferred 40082 clusters with 30 clusters populated
In stage 1: transferred 30819 clusters with 30 clusters populated
In stage 2: transferred 13890 clusters with 18 clusters populated
In stage 3: transferred 10801 clusters with 7 clusters populated
In stage 4: transferred 8700 clusters with 6 clusters populated
In stage 5: transferred 6823 clusters with 7 clusters populated
In stage 6: transferred 5738 clusters with 6 clusters populated
In stage 7: transferred 5130 clusters with 6 clusters populated
In stage 8: transferred 4522 clusters with 6 clusters populated
In stage 9: transferred 4381 clusters with 7 clusters populated
In stage 10: transferred 4358 clusters with 7 clusters populated
In stage 11: transferred 4486 clusters with 7 clusters populated
In stage 12: transferred 4501 clusters with 5 clusters populated
In stage 13: transferred 4486 clusters with 6 clusters populated
In stage 14: transferred 4441 clusters with 5 clusters populated
In stage 15: transferred 443

In [74]:
out_tuple = list(zip(y, in_sent))

In [75]:

for topic_no in set(y):
    print(f'Topic {topic_no}')
    counter = 1
    for i in out_tuple:
        if i[0]==topic_no and counter < 10:
            print(i[1])
            counter += 1

    print('')

Topic 0
Very good. very fragrant.
I hope it lasts because it is made of fur.
Delivery ahead of time. Very practical and easy to handle product.
Nor did the invoice
the product arrived before the deadline, and I really liked it !! now my 3 year old boy will make the most of it !!
The brushes came crumpled, not consistent with some photos, but consistent with the description. They are weak, do not fit in the case and a cool shaft
I loved it! Small, easy to use, does not scratch, appeals and cleans the skin .. recommend
N/a
Product easy to assemble and perfectly serves the purpose to optimize space.

Topic 1
Congratulations lannister stores loved shopping online safe and practical Congratulations to all happy Easter
I always shop at lannister. For me it&#39;s the best online store
nothing to declare
congratulations to store and dealer although it took a while
Very good pre workout, one of the best on the market, if not the best.
I recommend the shop I always buy at lannister.
Good night t

In [ ]:

# frequency per topic 
# frequency overall

out_sent

In [ ]:



count_vect2 = CountVectorizer(min_df=0.001, stop_words=stop_words)
doc_term_matrix2 = count_vect2.fit_transform([' '.join(i) for i in out_sent])

In [ ]:
pd.DataFrame({'a':doc_term_matrix2.todense().sum(axis=0).transpose()})



In [ ]:
count_vect2.get_feature_names() 

In [ ]:

frequencies = pd.DataFrame({'word':count_vect2.get_feature_names(), 'occurence_total':doc_term_matrix2.todense().sum(axis=0).tolist()[0]})

In [ ]:

frequencies.sort_values('occurence_total',ascending=False)

In [ ]:


len(y)

In [ ]:
doc_term_matrix2.shape

In [ ]:
y

In [ ]:
for idx, i in enumerate(y):
    print(idx)


In [ ]:

topic_no = 0

indexes_for_topic = []
for idx, i in enumerate(y):
    if i == topic_no:
        indexes_for_topic.append(idx)


In [ ]:

frequencies_topic = pd.DataFrame({'word':count_vect2.get_feature_names(), 'occurence_total':doc_term_matrix2[indexes_for_topic,:].todense().sum(axis=0).tolist()[0]})


In [ ]:

for topic_no in set(y):
    indexes_for_topic = []

    for idx, i in enumerate(y):
        if i == topic_no:
            indexes_for_topic.append(idx)


    frequencies_topic = pd.DataFrame({'word':count_vect2.get_feature_names(), 'occurence_total':doc_term_matrix2[indexes_for_topic,:].todense().sum(axis=0).tolist()[0]})
    frequencies['occurences_topic_'+str(topic_no)] = doc_term_matrix2[indexes_for_topic,:].todense().sum(axis=0).tolist()[0]

In [ ]:
def perc(x):
    return x/sum(x)

frequencies2 = frequencies.select_dtypes(include=['int64']).apply(perc)

In [ ]:
frequencies2['word'] = frequencies.word
frequencies2['occurence_total_num'] = frequencies.occurence_total

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")

In [ ]:
frequencies3=frequencies2.melt(value_vars=frequencies2.select_dtypes(['float']).columns, id_vars = 'word')


In [ ]:
frequencies3

In [ ]:


words_to_show = list(frequencies.sort_values('occurence_total',ascending=False).head(20).word)

In [ ]:
frequencies4=frequencies3[frequencies3.word.isin(words_to_show)]#.query('word in !words_to_show')

In [ ]:
g = sns.catplot(
    data=frequencies4, kind="bar",
    y="word", x="value", hue="variable",
    palette="dark", alpha=.6, height=6
)

In [ ]:
sns.catplot(
    data=frequencies4, kind="bar",
    y="word", x="value", hue="variable",
    palette="dark", alpha=.6, height=6
)

In [ ]:



count_vect_small = CountVectorizer(min_df=0.01, stop_words=stop_words)
doc_term_matrix_small = count_vect_small.fit_transform(df['message_en'].values)
doc_term_matrix_small

In [ ]:
from sklearn.decomposition import PCA, TruncatedSVD

In [ ]:
p = PCA(n_components=2)
p.fit(doc_term_matrix_small.todense())
p_data = p.transform(doc_term_matrix_small.todense())

In [ ]:
import matplotlib.pyplot as plt
p_data

In [ ]:
plt.plot(p_data[:,0], p_data[:,1], 'o')

In [ ]:


out = pd.DataFrame({"message_en":df['message_en'], 
'x1':p_data[:,0], 'x2':p_data[:,1]})

In [ ]:

out.to_csv('review_reduction.csv')

In [ ]:

# Very basic EDA of words in the corpus
df_occurences = df.copy()
df['message_en']
df['message_en'].str.contains('deadline')
# df['message_en'].str.contains('delivery')
# df['message_en'].str.contains('product')

# list(df['message_en'].str.split(' '))



In [ ]:
list(df['message_en'][100:120])




In [ ]:
import spacy
from nltk import Tree


en_nlp = spacy.load('en_core_web_sm')

doc = en_nlp(df['message_en'][110])

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_


[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

In [ ]:
df_occurences.assign(a = lambda x: x['message_en'].str.contains('deadline'))

## Random spacy 

In [ ]:


# Find subjects of sentences

import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

In [ ]:

sub_toks = [tok for tok in doc if (tok.dep_ == "nsubj") ]

print(sub_toks) 

In [ ]:

df.head()

In [ ]:

doc = nlp(list(df.message_en)[4])

In [ ]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

In [ ]:


for token in doc:
    print(token.lemma_, token.pos_)

In [ ]:

for i in list(df.message_en)[:30]:
    print(i)
    doc = nlp(i)
    for token in doc:
        if token.pos_ in ['VERB', 'ADJ', 'NOUN', 'ADV']:
            print(token.lemma_, token.pos_)
        # print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #     token.shape_, token.is_alpha, token.is_stop)

    print('')
